In [2]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import plotly.graph_objects as go
import plotly.io as pio
import schedule
import time

## Generate API key on Alpha Vantage website

In [43]:
api_key = '' # Fill the API key from Alpha Vantage

pio.renderers.default = 'browser'

## Candlestick Patterns defined

In [5]:
def is_doji(row, threshold=0.1):
        return abs(row['1. open'] - row['4. close']) <= threshold * (row['2. high'] - row['3. low'])

def is_tweezer(row1, row2, threshold=0.1):
    if row1['4. close'] > row1['1. open'] and row2['4. close'] < row2['1. open']:
        if row1['2. high'] >= row2['2. high'] and abs(row1['3. low'] - row2['3. low']) <= threshold * (row1['2. high'] - row1['3. low']):
            return True
    elif row1['4. close'] < row1['1. open'] and row2['4. close'] > row2['1. open']:
        if row1['3. low'] <= row2['3. low'] and abs(row1['2. high'] - row2['2. high']) <= threshold * (row1['2. high'] - row1['3. low']):
            return True
    return False

## Candlestick Patterns searched

In [31]:
def Doji_found(data):
    for i in range(len(data)):
        row = data.iloc[i]
        if is_doji(row, threshold=0.1):
            next_row = data.iloc[i - 1] if i > 0 else None
            if next_row is not None:
                if row['4. close'] > row['1. open'] and next_row['4. close'] > next_row['1. open']:
                    return 1, row.name
                elif row['4. close'] < row['1. open'] and next_row['4. close'] < next_row['1. open']:
                    return -1, row.name
    return 0, None

def Tweezer_found(data):
    for i in range(len(data) - 1):
        current_row = data.iloc[i]
        next_row = data.iloc[i - 1]
        if is_tweezer(current_row, next_row, threshold=0.1):
            if current_row['4. close'] > current_row['1. open']:
                return 1, current_row.name
            else:
                return -1, current_row.name
    return 0, None

## Data Processing

In [36]:
def fetch_and_process_data():
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_intraday(symbol='MSFT', interval='1min', outputsize='full')
    
    data_sorted = data.sort_index(ascending=False)
    latest_5_entries = data_sorted.head(5)
    candlestick_data = latest_5_entries[['1. open', '2. high', '3. low', '4. close']]
    
    fig = go.Figure(data=[go.Candlestick(
        x=candlestick_data.index,
        open=candlestick_data['1. open'],
        high=candlestick_data['2. high'],
        low=candlestick_data['3. low'],
        close=candlestick_data['4. close']
    )])
    
    fig.update_layout(
        title='MSFT Stock Price - Last 5 Minutes',
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        xaxis_rangeslider_visible=False
    )
    
    fig.show()

    doji_signal, doji_time = Doji_found(latest_5_entries)
    tweezer_signal, tweezer_time = Tweezer_found(latest_5_entries)
    
    if doji_signal == 1:
        print(f"Bullish Doji pattern detected on {doji_time}")
        print("Buy!")
        
    elif doji_signal == -1:
        print(f"Bearish Doji pattern detected on {doji_time}")
        print("Sell!")

    if tweezer_signal == 1:
        print(f"Bullish Tweezer pattern detected on {tweezer_time}")
        print("Buy!")
    elif tweezer_signal == -1:
        print(f"Bearish Tweezer pattern detected on {tweezer_time}")
        print("Sell!")

In [37]:
fetch_and_process_data()

Bullish Doji pattern detected on 2024-07-12 19:58:00
Buy!


## Automated to run the script every 1 minute. Close the process using Ctrl + C

In [ ]:
schedule.every(1).minutes.do(fetch_and_process_data)

try:
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    print("Script stopped by user.")